In [1]:
!pip install langchain chromadb transformers sentence-transformers pypdf langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107

## 랭체인(Langchain)을 이용한 PDF 전처리

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings

삼성전자 기업분석 PDF를 다운로드합니다.

In [3]:
!wget https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf

--2024-08-05 05:25:54--  https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf
Resolving wdr.ubion.co.kr (wdr.ubion.co.kr)... 61.100.182.43
Connecting to wdr.ubion.co.kr (wdr.ubion.co.kr)|61.100.182.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253369 (1.2M) [application/pdf]
Saving to: ‘gsat_170823.pdf’

gsat_170823.pdf     100%[===================>]   1.19M  1.81MB/s    in 0.7s    

2024-08-05 05:25:55 (1.81 MB/s) - ‘gsat_170823.pdf’ saved [1253369/1253369]



PDF를 PyPDFLoader로 로드하고 분할합니다. 해당 PDF는 참고로 27페이지입니다.

PDF에 있는 table을 그대로 가져오기는 힘듦
글로벌 서비스로 library 활용 필요

**저렴하게 변경하는 방법**
1. table이 있는 것을 캡쳐
2. 클로드 소나에게 table 내용을 말로 표현해달라고 요청
3. tr, td, colspan, rowspan에 대한 내용을 알려주는 prompt 작성하여 요청

In [7]:
## pdf 파일로드 하고 쪼개기
loader = PyPDFLoader('https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf')
pages = loader.load_and_split()
print(len(pages))

27


In [8]:
print(len(pages))

27


데이터의 형식은 다음을 따릅니다.  

`Document(page_content='내용', metadata={'source': 파일명, 'page': 페이지 번호})`

In [6]:
pages[0]

Document(metadata={'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'page': 0}, page_content='2\n01삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1 기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰 , 정보통신기기 , 반도체 , TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산 1) 244조 매출액 2) 200조\n임직원수 95,374명 \n∙1975년 1월 주식시장 상장\n∙1984년 2월 삼성전자공업주식회사 ->삼성전자주식회사로 사명 변경 \n∙CE(Consumer Electronics), IM(Information technology & Mobile co mmunications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영 .\n부문 제품\nCE TV, 모니터 , 냉장고 , 세탁기 , 에어컨 , 프린터 , 의료기기 등\nIM HHP, 네트워크시스템 , 컴퓨터 , 디지털카메라 등\nDS DRAM, NAND Flash, 모바일 AP, LCD패널, OLED패널, LED 등 \n∙주요 사업은 전자전지기계 등 제조 , 전자통신기 등 제도 , 컴퓨터 등 제조 , 반도체 제조 ·조립 등 . 주요 \n제품과 구성비율은 HHP, 네트워크시스템 , 컴퓨터 , 디지털카메라 등 54%, TV, 모니터 , 냉장고 , 세탁기 , \n에어컨 등 2 4 % ,  D R A M ,  N A N D  F l a s h ,  모바일 AP 등 19% 등으로 구분 \n∙지역별로는 본사를 거점으로 한국 및 CE, IM 부문 산하 해외 9개 지역총괄과 DS 부문 산하 해외 5개 \n지

길이 500기준으로 쪼개줍니다.

In [9]:
## chunk로 쪼개기
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splited_docs = text_splitter.split_documents(pages)
print(len(splited_docs))

69


In [11]:
splited_docs[0]

Document(metadata={'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'page': 0}, page_content='2\n01삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1 기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰 , 정보통신기기 , 반도체 , TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산 1) 244조 매출액 2) 200조\n임직원수 95,374명 \n∙1975년 1월 주식시장 상장\n∙1984년 2월 삼성전자공업주식회사 ->삼성전자주식회사로 사명 변경 \n∙CE(Consumer Electronics), IM(Information technology & Mobile co mmunications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영 .\n부문 제품\nCE TV, 모니터 , 냉장고 , 세탁기 , 에어컨 , 프린터 , 의료기기 등')

실제로 길이를 재보면 500이 넘지 않는 것을 확인할 수 있습니다.

In [12]:
len(splited_docs[0].page_content)

498

오픈소스 임베딩을 사용해보겠습니다.

OpenAI의 Ada도 사용 가능하나 설치해서 사용할 때는 북경대 AI Labs 사용 가능

내부 통신으로 해결해야할 경우 유용함

In [14]:
model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')

## 크로마(Chroma)

다음은 Langchain에서 Chroma를 이용하여 벡터 DB 사용 시에 가장 간단한 사용법입니다.

```python
# Chroma 를 통해 벡터 저장소 생성. OpenAIEmbeddings()를 사용하거나 HuggingFaceEmbeddings()를 사용하면 된다.
chroma_db = Chroma.from_documents(splited_docs, OpenAIEmbeddings())

# 유사도 검색(쿼리)
similar_docs = chroma_db.similarity_search("사용자의 질문")
```

실제로 실행해봅시다.

In [15]:
## Chroma 기반 pdf(docs 벡터화)
db = Chroma.from_documents(splited_docs, model_huggingface)

print('문서의 수:', db._collection.count())

문서의 수: 69


In [18]:
## 질의하기
question = '삼성전자의 주요 사업영역은?'
docs = db.similarity_search(question)

In [19]:
len(docs)

4

어떤 embedding을 사용하느냐에 따라 결과가 다르게 나오므로 여러가지를 사용하며 비교해볼 필요 있음

In [20]:
for doc in docs:
  print(doc)
  print('--' * 100)

page_content='11
Ⅱ 기업 상세 분석
1 사업분야 (내용)
Q1삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?
A
삼성전자는 크게 CE(Consumer Electronics) 사업부문 , IM(Information technology & Mobile communica-
tion) 사업부문 , DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다 .
⑴ Consumer Electronics (CE) 부문 
① 영상디스플레이 : 진화하는 스마트 TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 
도전
② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비
③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발
⑵ Information technology & Mobile communication (IM) 부문' metadata={'page': 9, 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf'}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
page_content='2
01삼성전자 기업분석
(Samsung Electronics Co., Ltd)
Ⅰ 기업 일반 
1 기업개요
1) 기업소개 
본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)
사업분야 삼성그룹의 대표 기업으로 휴대폰 , 정보통신기기 , 반도체 , TV 등을 생산 판매하는 제조업체
홈페이지 www.samsung.com/sec 구분 전기전자 대기업  
설립일 1961년 07월

vectorize된 텍스트를 Chroma db 파일로 저장하기

In [21]:
db_toFiles = Chroma.from_documents(splited_docs, model_huggingface, persist_directory = './samsumg.db')
# db_toFiles.delete_collection()
print('문서의 수:', db_toFiles._collection.count())

문서의 수: 69


In [ ]:
## 질의하기
question = '삼성전자의 주요 사업영역은?'
docs = db_toFiles.similarity_search(question)
print(len(docs))
docs[0]

4


Document(page_content='11\nⅡ 기업 상세 분석\n1 사업분야 (내용)\nQ1삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n삼성전자는 크게 CE(Consumer Electronics) 사업부문 , IM(Information technology & Mobile communica-\ntion) 사업부문 , DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다 .\n⑴ Consumer Electronics (CE) 부문 \n① 영상디스플레이 : 진화하는 스마트 TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 \n도전\n② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비\n③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발\n⑵ Information technology & Mobile communication (IM) 부문', metadata={'page': 9, 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf'})

In [ ]:
%ls -al

total 1244
drwxr-xr-x 1 root root    4096 Jun 27 13:41 ./
drwxr-xr-x 1 root root    4096 Jun 27 13:38 ../
drwxr-xr-x 4 root root    4096 Jun 25 17:39 .config/
-rw-r--r-- 1 root root 1253369 Aug 23  2017 gsat_170823.pdf
drwxr-xr-x 1 root root    4096 Jun 25 17:40 sample_data/
drwxr-xr-x 3 root root    4096 Jun 27 13:41 samsumg.db/


 db 파일을 읽어와서 바로 모델에 적용하여 결과를 볼 수 있습니다.


 - 로딩없이 POC 용도로 사용

In [22]:
db_fromfile = Chroma(persist_directory = './samsumg.db',embedding_function=model_huggingface)
print('문서의 수:', db_fromfile._collection.count())

## 질의하기
question = '삼성전자의 주요 사업영역은?'
docs = db_fromfile.similarity_search(question)
print(len(docs))
docs[0]

문서의 수: 69
4


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


Document(metadata={'page': 9, 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf'}, page_content='11\nⅡ 기업 상세 분석\n1 사업분야 (내용)\nQ1삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n삼성전자는 크게 CE(Consumer Electronics) 사업부문 , IM(Information technology & Mobile communica-\ntion) 사업부문 , DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다 .\n⑴ Consumer Electronics (CE) 부문 \n① 영상디스플레이 : 진화하는 스마트 TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 \n도전\n② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비\n③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발\n⑵ Information technology & Mobile communication (IM) 부문')

In [ ]:
question = '삼성전자의 주요 사업영역은?'

# TOP 3개만 추출
docs3 = db_fromfile.similarity_search_with_relevance_scores(question, k=3)

In [ ]:
for doc in docs3:
  print(doc)
  print('--' * 100)

(Document(page_content='11\nⅡ 기업 상세 분석\n1 사업분야 (내용)\nQ1삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n삼성전자는 크게 CE(Consumer Electronics) 사업부문 , IM(Information technology & Mobile communica-\ntion) 사업부문 , DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다 .\n⑴ Consumer Electronics (CE) 부문 \n① 영상디스플레이 : 진화하는 스마트 TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 \n도전\n② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비\n③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발\n⑵ Information technology & Mobile communication (IM) 부문', metadata={'page': 9, 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf'}), 0.6592675235345931)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
(Document(page_content='2\n01삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1 기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰 , 정보통신기기 , 반도체 , TV 등을 생산 

In [ ]:
print(f"가장 유사한 문서 : {docs3[0][0].page_content}")
print('*'*20)
print(f"가장 유사한 문서의 문서의 유사도 : {docs3[0][1]}")

가장 유사한 문서 : 11
Ⅱ 기업 상세 분석
1 사업분야 (내용)
Q1삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?
A
삼성전자는 크게 CE(Consumer Electronics) 사업부문 , IM(Information technology & Mobile communica-
tion) 사업부문 , DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다 .
⑴ Consumer Electronics (CE) 부문 
① 영상디스플레이 : 진화하는 스마트 TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 
도전
② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비
③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발
⑵ Information technology & Mobile communication (IM) 부문
********************
가장 유사한 문서의 문서의 유사도 : 0.6592675235345931


## 크로마 문서 추가

In [ ]:
for key in db_fromfile._collection.get():
  print(key)

ids
embeddings
metadatas
documents
uris
data
included


In [ ]:
# id를 출력하되 상위 5개만 출력
db_fromfile._collection.get()['ids'][:5]

['0219a4e8-8d85-490b-81bb-8f15f8662250',
 '05fbbc5c-c734-4872-b19a-c7194b57b36e',
 '06b21b95-3b21-4ab2-91fa-dacda24a40ca',
 '0d903cc7-77a5-4d9d-9522-eb4804b59020',
 '1281aff2-020b-4e73-b5d5-83b53b4948bb']

In [ ]:
print('현재 문서 수 :', len(db_fromfile._collection.get()['ids']))

현재 문서 수 : 69


In [ ]:
# 신규 문서 추가 하기
text1 = '''삼성전자가 미국 반도체업체 엔비디아에 고대역폭 메모리(HBM)를 납품하기 위한 테스트를 아직 통과하지 못했다고 로이터통신이 3명의 익명 소식통을 인용해 24일 보도했다.'''
text2 = '''엔비디아발 훈풍으로 전 세계 반도체가 들썩이는 가운데 국내 증시에서는 SK하이닉스 (198,600원 ▼1,400 -0.70%)의 활약이 돋보인다. 엔비디아에 대한 투자자들의 기대치가 높아지고 있어 SK하이닉스 역시 상승세를 이어갈 것이란 전망이 쏟아진다. 다만 SK하이닉스 주가에 상승동력으로 작용했던 엔비디아 실적 재료가 소진된 점은 차익실현 매물이 출현할 가능성을 높인다.'''

In [ ]:
from langchain.docstore.document import Document

new_doc = Document(
    page_content=text2,
    metadata={
        "source": "wareconn.com",
        "page": 1
    }
)

In [ ]:
new_docs = [new_doc]

id = db_fromfile.add_documents(
    new_docs,
    ids=["7"]
)

In [ ]:
id

['7']

In [ ]:
print('현재 문서 수 :', len(db_fromfile._collection.get()['ids']))

현재 문서 수 : 70


## 크로마 문서 삭제

In [ ]:
# delete the last document
print("count before", db_fromfile._collection.count())
db_fromfile._collection.delete(ids=['25dd1dba-78fa-4b4d-99ee-2aa05baaa069'])
print("count after", db_fromfile._collection.count())

count before 70
count after 70


## **---------------------**

## 파이스(Faiss)

In [ ]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 48.8 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import FAISS

다음은 Langchain에서 FAISS를 이용하여 벡터 DB 사용 시에 가장 간단한 사용법입니다.

```python
# FAISS 를 통해 벡터 저장소 생성. OpenAIEmbeddings()를 사용하거나 HuggingFaceEmbeddings()를 사용하면 된다.
faiss_db = FAISS.from_documents(splited_docs, OpenAIEmbeddings())

# 유사도 검색(쿼리)
similar_docs = faiss_db.similarity_search("사용자의 질문")
```

In [ ]:
faiss_db = FAISS.from_documents(splited_docs, model_huggingface)
question = '삼성전자의 주요 사업영역은?'

docs = faiss_db.similarity_search(question)

In [ ]:
len(docs)

4

In [ ]:
for doc in docs:
  print(doc)
  print('--' * 100)

page_content='11\nⅡ 기업 상세 분석\n1 사업분야 (내용)\nQ1삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n삼성전자는 크게 CE(Consumer Electronics) 사업부문 , IM(Information technology & Mobile communica-\ntion) 사업부문 , DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다 .\n⑴ Consumer Electronics (CE) 부문 \n① 영상디스플레이 : 진화하는 스마트 TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 \n도전\n② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비\n③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발\n⑵ Information technology & Mobile communication (IM) 부문' metadata={'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'page': 9}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
page_content='2\n01삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1 기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰 , 정보통신기기 , 반도체 , TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자

You can also save and load a FAISS index. This is useful so you don't have to recreate it everytime you use it.



In [ ]:
# 파일 저장
faiss_db.save_local('faiss_index')

## 파일 불러오기
new_db_faiss = FAISS.load_local('faiss_index', model_huggingface, allow_dangerous_deserialization=True)
question = '삼성전자의 주요 사업영역은?'

docs = new_db_faiss.similarity_search(question)

In [ ]:
for doc in docs:
  print(doc)
  print('--' * 100)

page_content='11\nⅡ 기업 상세 분석\n1 사업분야 (내용)\nQ1삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n삼성전자는 크게 CE(Consumer Electronics) 사업부문 , IM(Information technology & Mobile communica-\ntion) 사업부문 , DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다 .\n⑴ Consumer Electronics (CE) 부문 \n① 영상디스플레이 : 진화하는 스마트 TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 \n도전\n② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비\n③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발\n⑵ Information technology & Mobile communication (IM) 부문' metadata={'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'page': 9}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
page_content='2\n01삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1 기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰 , 정보통신기기 , 반도체 , TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자

In [ ]:
print('저장된 문서 개수:', faiss_db.index.ntotal)

저장된 문서 개수: 69


In [ ]:
# 저장된 문서들 출력
faiss_db.docstore._dict

{'a4604038-eb98-44f9-931c-4317d5b53226': Document(page_content='2\n01삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1 기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰 , 정보통신기기 , 반도체 , TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산 1) 244조 매출액 2) 200조\n임직원수 95,374명 \n∙1975년 1월 주식시장 상장\n∙1984년 2월 삼성전자공업주식회사 ->삼성전자주식회사로 사명 변경 \n∙CE(Consumer Electronics), IM(Information technology & Mobile co mmunications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영 .\n부문 제품\nCE TV, 모니터 , 냉장고 , 세탁기 , 에어컨 , 프린터 , 의료기기 등', metadata={'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'page': 0}),
 'dd91651d-4d7b-4f54-bf22-a8a787d879ae': Document(page_content='IM HHP, 네트워크시스템 , 컴퓨터 , 디지털카메라 등\nDS DRAM, NAND Flash, 모바일 AP, LCD패널, OLED패널, LED 등 \n∙주요 사업은 전자전지기계 등 제조 , 전자통신기 등 제도 , 컴퓨터 등 제조 , 반도체 제조 ·조립 등 . 주요 \n제품과 구성비율은 HHP, 네트워크시스템 , 컴퓨터 , 디지털카메라 등 54%, TV, 모니터 , 냉장고 , 세탁기 , \n에어컨 등 2 4 % ,  D R A M ,  N

## 파이스 문서 삭제

In [ ]:
# 기존 문서 삭제하기. 삭제 시에는 id값으로 한다.
print("count before:", faiss_db.index.ntotal)

faiss_db.delete(['649142b0-8aee-4147-a205-7efcfa36d63f'])

print("count after:", faiss_db.index.ntotal)

count before: 69


ValueError: Some specified ids do not exist in the current store. Ids not found: {'649142b0-8aee-4147-a205-7efcfa36d63f'}

## 파이스 문서 추가

In [ ]:
# 신규 문서 추가 하기
text1 = '''삼성전자가 미국 반도체업체 엔비디아에 고대역폭 메모리(HBM)를 납품하기 위한 테스트를 아직 통과하지 못했다고 로이터통신이 3명의 익명 소식통을 인용해 24일 보도했다.'''
text2 = '''엔비디아발 훈풍으로 전 세계 반도체가 들썩이는 가운데 국내 증시에서는 SK하이닉스 (198,600원 ▼1,400 -0.70%)의 활약이 돋보인다. 엔비디아에 대한 투자자들의 기대치가 높아지고 있어 SK하이닉스 역시 상승세를 이어갈 것이란 전망이 쏟아진다. 다만 SK하이닉스 주가에 상승동력으로 작용했던 엔비디아 실적 재료가 소진된 점은 차익실현 매물이 출현할 가능성을 높인다.'''

In [ ]:
new_db = FAISS.from_texts([text1, text2], model_huggingface)

In [ ]:
print('신규 DB 샘플 개수:', new_db.index.ntotal)

In [ ]:
# 저장된 문서들 출력
new_db.docstore._dict

In [ ]:
faiss_db.merge_from(new_db)

In [ ]:
print("count after merge:", faiss_db.index.ntotal)